# Plot ECCOv4r5 Time-Varying Runoff Run

In [119]:
# tell Python to use the ecco_v4_py in the 'ECCOv4-py' repository
from os.path import join,expanduser
import sys

# identify user's home directory
user_home_dir = expanduser('~')

# import the ECCOv4 py library 
sys.path.insert(0,join(user_home_dir,'ECCOv4-py'))
import ecco_v4_py as ecco

## import required packages
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import glob
import pathlib as Path
import os
import re
import cmocean
import matplotlib.cm as cm

In [13]:
input_dir = '/efs_ecco/mzahn/r5/WORKINGDIR/ECCOV4/release5/run/diags/SALT_daily_mean/'
pattern = os.path.join(input_dir, 'SALT_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

In [17]:
input_dir = '/efs_ecco/mzahn/r5/WORKINGDIR/ECCOV4/release5/run/diags/SALT_daily_mean/'

# Use glob to find all files matching pattern in runoff_dir
# Adjust the pattern as needed to match your filenames, e.g., all years starting with 'jra55_do_runoff_ECCO_V4r5_'
file_pattern = os.path.join(input_dir, 'SALT_daily_mean.*.data')
file_list = sorted(glob.glob(file_pattern))

datasets = []

for filepath in file_list[0:10]:
    # Extract the filename without extension to pass to read_llc_to_tiles
    fname = os.path.basename(filepath)
    
    # Read the runoff data using ECCO functions
    data = ecco.read_llc_to_tiles(input_dir, fname, nl=12, less_output = True)
    
    # Convert tiles to xarray DataArray, specify time as 5th dimension
    ds = ecco.llc_tiles_to_xda(data, var_type='c', dim4='depth', dim5='time')
    
    # Assign a year coordinate from filename (assuming year is in the filename, e.g., last 4 digits)
    # Adjust this parsing to fit your actual filename structure
    year_str = fname.split('_')[-1]
    try:
        year = int(year_str)
    except ValueError:
        year = None
    
    if year is not None:
        ds = ds.expand_dims({'year':[year]})
    
    datasets.append(ds)

# Concatenate all yearly datasets along the 'year' dimension
salt_all_years = xr.concat(datasets, dim='year')

In [133]:
from datetime import timedelta, datetime

def date_to_iter_number(date,seconds_per_iter = 3600):
    total_seconds = (date-datetime(1992,1,1)).total_seconds()
    iter_number = total_seconds/seconds_per_iter
    return(iter_number)

def iter_number_to_date(iter_number,seconds_per_iter=3600):
    total_seconds = iter_number*seconds_per_iter
    date = datetime(1992,1,1) + timedelta(seconds=total_seconds)
    return(date)

## Loading Salinity gifs

In [100]:
# ---------------------------------------------------------------
# Check november dates -- can change to your year as required
# ---------------------------------------------------------------
from datetime import timedelta, datetime

oct_2014_start = date_to_iter_number(datetime(2019,10,1))
dec_2014_end = date_to_iter_number(datetime(2019,12,31))

In [101]:
# --------------------------------------------
# Load in three months of data -- November 2015
# --------------------------------------------

geom = xr.open_dataset('/efs_ecco/ECCO/V4/r5/netcdf/native/geometry/GRID_GEOMETRY_ECCO_V4r5_native_llc0090.nc')

input_dir = '/efs_ecco/mzahn/r5/WORKINGDIR/ECCOV4/release5/run/diags/SALT_daily_mean'
pattern = os.path.join(input_dir, 'SALT_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_2014_start
end_num = dec_2014_end

salt_DA_list = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            salt_test = ecco.read_llc_to_tiles(input_dir, filename, llc=90, skip=0, nk=50, less_output= True);
            salt_test = np.where(geom.hFacC == 1, salt_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            salt_DA = xr.DataArray(
                salt_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            salt_DA_list.append(salt_DA);

# Concatenate all valid files
salt_OND2014 = xr.concat(salt_DA_list, dim='time');

In [102]:
import imageio

# Assume salt_all is your DataArray: (time, j, i)
# If it's 4D (e.g., time, k, j, i), select a level:
data = salt_OND2014.isel(k=0, tile=6)

# Directory for temporary frames
tmp_dir = 'tmp_frames'
os.makedirs(tmp_dir, exist_ok=True)

filenames = []

# Loop through time steps
for t in range(len(data.time)):  # or len(data.time) to use all
    fig, ax = plt.subplots(figsize=(6, 5))
    im = ax.imshow(data.isel(time=t).values, origin='lower', cmap=cmocean.cm.haline, vmin=27, vmax=34)
    ax.set_title(str(data.time[t].values))  # Use time label
    fig.colorbar(im, ax=ax)
    
    filename = os.path.join(tmp_dir, f'frame_{t:03d}.png')
    plt.savefig(filename)
    plt.close()
    filenames.append(filename)

# Create GIF
gif_filename = 'salt_OND2019.gif'
with imageio.get_writer(gif_filename, mode='I', duration=0.3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Cleanup
for filename in filenames:
    os.remove(filename)
os.rmdir(tmp_dir)

print(f"GIF saved to {gif_filename}")

/tmp/ipykernel_658/3133511560.py:29: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


GIF saved to salt_OND2019.gif


## Loading SIA gifs

In [103]:
# ---------------------------------------------------------------
# Check november dates -- can change to your year as required
# ---------------------------------------------------------------
from datetime import timedelta, datetime

oct_2014_start = date_to_iter_number(datetime(2014,10,1))
dec_2014_end = date_to_iter_number(datetime(2014,12,31))

In [104]:
# --------------------------------------------
# Load in three months of data -- November 2014
# --------------------------------------------

geom = xr.open_dataset('/efs_ecco/ECCO/V4/r5/netcdf/native/geometry/GRID_GEOMETRY_ECCO_V4r5_native_llc0090.nc')

input_dir = '/efs_ecco/mzahn/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIarea_daily_mean'
pattern = os.path.join(input_dir, 'SIarea_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_2014_start
end_num = dec_2014_end

siarea_DA_list = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            siarea_test = ecco.read_llc_to_tiles(input_dir, filename, llc=90, skip=0, nk=1, less_output=True);
            siarea_test = np.where(geom.hFacC.isel(k=0) == 1, siarea_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            time = iter_number_to_date(number)

            siarea_DA = xr.DataArray(
                siarea_test.reshape(1, 13, 90, 90),
                coords={'time': time, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            siarea_DA_list.append(siarea_DA);

# Concatenate all valid files
siarea_OND2014 = xr.concat(siarea_DA_list, dim='time');

In [105]:
import imageio

# Assume salt_all is your DataArray: (time, j, i)
# If it's 4D (e.g., time, k, j, i), select a level:
data = siarea_OND2014.isel(k=0, tile=6)

# Directory for temporary frames
tmp_dir = 'tmp_frames'
os.makedirs(tmp_dir, exist_ok=True)

filenames = []

# Loop through time steps
for t in range(len(data.time)):  # or len(data.time) to use all
    fig, ax = plt.subplots(figsize=(6, 5))
    cmap = cmocean.cm.ice.copy()
    cmap.set_bad(color='gray')
    im = ax.imshow(data.isel(time=t).values, origin='lower', cmap=cmap, vmin=0, vmax=1)
    ax.set_title(str(data.time[t].values))  # Use time label
    fig.colorbar(im, ax=ax)
    
    filename = os.path.join(tmp_dir, f'frame_{t:03d}.png')
    plt.savefig(filename)
    plt.close()
    filenames.append(filename)

# Create GIF
gif_filename = 'siarea_OND2014.gif'
with imageio.get_writer(gif_filename, mode='I', duration=0.3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Cleanup
for filename in filenames:
    os.remove(filename)
os.rmdir(tmp_dir)

print(f"GIF saved to {gif_filename}")

/tmp/ipykernel_658/3091898531.py:31: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


GIF saved to siarea_OND2019.gif


## Look at difference between time varying SALT and SIarea from control

### SALT

In [177]:
# ---------------------------------------------------------------
# Check november dates -- can change to your year as required
# ---------------------------------------------------------------
from datetime import timedelta, datetime

oct_2014_start = date_to_iter_number(datetime(2019,10,1))
dec_2014_end = date_to_iter_number(datetime(2019,12,31))

In [178]:
# --------------------------------------------
# Load in three months of data -- November 2014
# --------------------------------------------

geom = xr.open_dataset('/efs_ecco/ECCO/V4/r5/netcdf/native/geometry/GRID_GEOMETRY_ECCO_V4r5_native_llc0090.nc')

input_dir = '/efs_ecco/mzahn/r5/WORKINGDIR/ECCOV4/release5/run/diags/SALT_daily_mean'
pattern = os.path.join(input_dir, 'SALT_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_2014_start
end_num = dec_2014_end

salt_DA_list = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            salt_test = ecco.read_llc_to_tiles(input_dir, filename, llc=90, skip=0, nk=1, less_output=True);
            salt_test = np.where(geom.hFacC.isel(k=0) == 1, salt_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            time = iter_number_to_date(number)

            salt_DA = xr.DataArray(
                salt_test.reshape(1, 13, 90, 90),
                coords={'time': time, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            salt_DA_list.append(salt_DA);

# Concatenate all valid files
salt_OND2014 = xr.concat(salt_DA_list, dim='time');

In [179]:
# --------------------------------------------
# Load in three months of data -- November 2014
# --------------------------------------------

geom = xr.open_dataset('/efs_ecco/ECCO/V4/r5/netcdf/native/geometry/GRID_GEOMETRY_ECCO_V4r5_native_llc0090.nc')

input_dir = '/efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SALT_daily_mean'
pattern = os.path.join(input_dir, 'SALT_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_2014_start
end_num = dec_2014_end

salt_DA_list = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            salt_test = ecco.read_llc_to_tiles(input_dir, filename, llc=90, skip=0, nk=1, less_output=True);
            salt_test = np.where(geom.hFacC.isel(k=0) == 1, salt_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            time = iter_number_to_date(number)

            salt_DA = xr.DataArray(
                salt_test.reshape(1, 13, 90, 90),
                coords={'time': time, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            salt_DA_list.append(salt_DA);

# Concatenate all valid files
salt_control_OND2014 = xr.concat(salt_DA_list, dim='time');

In [180]:
import imageio

# Assume salt_all is your DataArray: (time, j, i)
# If it's 4D (e.g., time, k, j, i), select a level:
data_base = salt_control_OND2014.isel(k=0, tile=6)
data_vary = salt_OND2014.isel(k=0, tile=6)

# Directory for temporary frames
tmp_dir = 'tmp_frames'
os.makedirs(tmp_dir, exist_ok=True)

filenames = []

# Loop through time steps
for t in range(len(data_base.time)):  # or len(data.time) to use all
    fig, ax = plt.subplots(figsize=(6, 5))
    cmap = cm.get_cmap('bwr').copy()
    cmap.set_bad(color='gray')
    im = ax.imshow((data_vary.isel(time=t).values - data_base.isel(time=t).values), origin='lower', cmap=cmap, vmin=-0.5, vmax=0.5)
    ax.set_title(str(data_base.time[t].values))  # Use time label
    fig.colorbar(im, ax=ax)
    
    filename = os.path.join(tmp_dir, f'frame_{t:03d}.png')
    plt.savefig(filename)
    plt.close()
    filenames.append(filename)

# Create GIF
gif_filename = 'salt_time_vary_control_OND2019.gif'
with imageio.get_writer(gif_filename, mode='I', duration=0.3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Cleanup
for filename in filenames:
    os.remove(filename)
os.rmdir(tmp_dir)

print(f"GIF saved to {gif_filename}")

/tmp/ipykernel_658/2004219731.py:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('bwr').copy()
/tmp/ipykernel_658/2004219731.py:32: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


GIF saved to salt_time_vary_control_OND2019.gif


### SIarea

In [150]:
# ---------------------------------------------------------------
# Check november dates -- can change to your year as required
# ---------------------------------------------------------------
from datetime import timedelta, datetime

oct_2014_start = date_to_iter_number(datetime(2014,10,1))
dec_2014_end = date_to_iter_number(datetime(2014,12,31))

In [151]:
# --------------------------------------------
# Load in three months of data -- November 2014
# --------------------------------------------

geom = xr.open_dataset('/efs_ecco/ECCO/V4/r5/netcdf/native/geometry/GRID_GEOMETRY_ECCO_V4r5_native_llc0090.nc')

input_dir = '/efs_ecco/mzahn/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIarea_daily_mean'
# input_dir = '/efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIarea_daily_mean'
pattern = os.path.join(input_dir, 'SIarea_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_2014_start
end_num = dec_2014_end

siarea_DA_list = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            siarea_test = ecco.read_llc_to_tiles(input_dir, filename, llc=90, skip=0, nk=1, less_output=True);
            siarea_test = np.where(geom.hFacC.isel(k=0) == 1, siarea_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            time = iter_number_to_date(number)

            siarea_DA = xr.DataArray(
                siarea_test.reshape(1, 13, 90, 90),
                coords={'time': time, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            siarea_DA_list.append(siarea_DA);

# Concatenate all valid files
siarea_OND2014 = xr.concat(siarea_DA_list, dim='time');

In [152]:
# --------------------------------------------
# Load in three months of data -- November 2014
# --------------------------------------------

geom = xr.open_dataset('/efs_ecco/ECCO/V4/r5/netcdf/native/geometry/GRID_GEOMETRY_ECCO_V4r5_native_llc0090.nc')

# input_dir = '/efs_ecco/mzahn/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIarea_daily_mean'
input_dir = '/efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIarea_daily_mean'
pattern = os.path.join(input_dir, 'SIarea_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_2014_start
end_num = dec_2014_end

siarea_DA_list = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            siarea_test = ecco.read_llc_to_tiles(input_dir, filename, llc=90, skip=0, nk=1, less_output=True);
            siarea_test = np.where(geom.hFacC.isel(k=0) == 1, siarea_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            time = iter_number_to_date(number)

            siarea_DA = xr.DataArray(
                siarea_test.reshape(1, 13, 90, 90),
                coords={'time': time, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            siarea_DA_list.append(siarea_DA);

# Concatenate all valid files
siarea_control_OND2014 = xr.concat(siarea_DA_list, dim='time');

In [153]:
import imageio

# Assume salt_all is your DataArray: (time, j, i)
# If it's 4D (e.g., time, k, j, i), select a level:
data_base = siarea_control_OND2014.isel(k=0, tile=6)
data_vary = siarea_OND2014.isel(k=0, tile=6)

# Directory for temporary frames
tmp_dir = 'tmp_frames'
os.makedirs(tmp_dir, exist_ok=True)

filenames = []

# Loop through time steps
for t in range(len(data_base.time)):  # or len(data.time) to use all
    fig, ax = plt.subplots(figsize=(6, 5))
    cmap = cm.get_cmap('PiYG').copy()
    cmap.set_bad(color='gray')
    im = ax.imshow((data_vary.isel(time=t).values - data_base.isel(time=t).values), origin='lower', cmap=cmap, vmin=-0.05, vmax=0.05)
    ax.set_title(str(data_base.time[t].values))  # Use time label
    fig.colorbar(im, ax=ax)
    
    filename = os.path.join(tmp_dir, f'frame_{t:03d}.png')
    plt.savefig(filename)
    plt.close()
    filenames.append(filename)

# Create GIF
gif_filename = 'siarea_time_vary_control_OND2014.gif'
with imageio.get_writer(gif_filename, mode='I', duration=0.3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Cleanup
for filename in filenames:
    os.remove(filename)
os.rmdir(tmp_dir)

print(f"GIF saved to {gif_filename}")

/tmp/ipykernel_658/140385635.py:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('PiYG').copy()
/tmp/ipykernel_658/140385635.py:32: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


GIF saved to siarea_time_vary_control_OND2019.gif
